In [ ]:


año, torneo, nombre, edad,altura, ranking_ATP_inicial, minutos_jugados_acumulado_hist,
pj_acum_historicos, pg_acum_historico, pp_acum_historico, titulos_ganados_acum, ultima_modificacion, posicion_final, score


2014,  roland_garros, rafa, 7, 1, 7, 0, sum(minutos), fijo, 6, clay, 1, 100, 2





predecir 2015

1 -> 32/32=1
2 -> 31/32=0.99

In [1]:
# Use this initial code to work in the notebook as if it were a module, that 
# is, to be able to export classes and functions from other subpackages.

import pyprojroot
import sys

package_path = pyprojroot.here().__str__()
if package_path not in sys.path:
    sys.path.append(package_path)

In [3]:
clean_data = 'atp_matches_historic.csv'

In [16]:
from utils import LoadData
import pandas as pd

In [234]:
df = LoadData().from_csv(file_name=clean_data)
df.head(2)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,winner_rank,winner_rank_points,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,loser_rank,loser_rank_points,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
0,2000-560,US Open,Hard,128.0,G,20000828.0,1.0,101736.0,1.0,NaN,Andre Agassi,R,180.0,USA,30.332649,1.0,3754.0,103297.0,NaN,WC,Alex Kim,R,175.0,USA,21.689254,432.0,53.0,6-4 6-2 6-0,5.0,R128,89.0,8.0,4.0,80.0,50.0,37.0,16.0,12.0,2.0,3.0,3.0,8.0,90.0,52.0,30.0,13.0,12.0,5.0,12.0
1,2000-560,US Open,Hard,128.0,G,20000828.0,2.0,103096.0,NaN,NaN,Arnaud Clement,R,173.0,FRA,22.696783,37.0,928.0,103292.0,NaN,NaN,Gaston Gaudio,R,175.0,ARG,21.719370,26.0,1067.0,6-3 6-4 6-2,5.0,R128,105.0,11.0,5.0,92.0,58.0,44.0,16.0,14.0,2.0,3.0,5.0,4.0,84.0,53.0,34.0,12.0,13.0,7.0,12.0


In [235]:
df.tourney_date = pd.to_datetime(df.tourney_date.astype(int), format="%Y%m%d")
df.head(2)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,winner_rank,winner_rank_points,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,loser_rank,loser_rank_points,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
0,2000-560,US Open,Hard,128.0,G,2000-08-28,1.0,101736.0,1.0,NaN,Andre Agassi,R,180.0,USA,30.332649,1.0,3754.0,103297.0,NaN,WC,Alex Kim,R,175.0,USA,21.689254,432.0,53.0,6-4 6-2 6-0,5.0,R128,89.0,8.0,4.0,80.0,50.0,37.0,16.0,12.0,2.0,3.0,3.0,8.0,90.0,52.0,30.0,13.0,12.0,5.0,12.0
1,2000-560,US Open,Hard,128.0,G,2000-08-28,2.0,103096.0,NaN,NaN,Arnaud Clement,R,173.0,FRA,22.696783,37.0,928.0,103292.0,NaN,NaN,Gaston Gaudio,R,175.0,ARG,21.719370,26.0,1067.0,6-3 6-4 6-2,5.0,R128,105.0,11.0,5.0,92.0,58.0,44.0,16.0,14.0,2.0,3.0,5.0,4.0,84.0,53.0,34.0,12.0,13.0,7.0,12.0


In [253]:
sub_df = df[(df.tourney_name == 'US Open') & (df.tourney_date.dt.year == 2015)].copy()
sub_df.shape

(127, 49)

In [35]:
sub_df.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,winner_rank,winner_rank_points,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,loser_rank,loser_rank_points,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
7747,2015-560,US Open,Hard,128.0,G,2015-08-31,1.0,104925.0,1.0,NaN,Novak Djokovic,R,188.0,SRB,28.276523,1.0,14865.0,105154.0,NaN,NaN,Joao Souza,R,193.0,BRA,27.260780,91.0,554.0,6-1 6-1 6-1,5.0,R128,NaN,8.0,0.0,52.0,39.0,36.0,8.0,11.0,0.0,0.0,3.0,2.0,57.0,31.0,17.0,7.0,10.0,3.0,10.0
7748,2015-560,US Open,Hard,128.0,G,2015-08-31,2.0,104890.0,NaN,NaN,Andreas Haider Maurer,R,190.0,AUT,28.443532,52.0,870.0,105577.0,NaN,NaN,Vasek Pospisil,R,193.0,CAN,25.188227,45.0,1000.0,6-4 3-6 6-7(0) 6-0 6-1,5.0,R128,NaN,12.0,10.0,120.0,61.0,50.0,35.0,20.0,3.0,4.0,24.0,16.0,186.0,110.0,79.0,28.0,22.0,19.0,25.0
7749,2015-560,US Open,Hard,128.0,G,2015-08-31,3.0,104559.0,NaN,NaN,Teymuraz Gabashvili,R,188.0,RUS,30.272416,53.0,867.0,104665.0,NaN,NaN,Pablo Andujar,R,180.0,ESP,29.601643,49.0,945.0,7-6(6) 0-6 4-6 6-4 RET,5.0,R128,NaN,7.0,5.0,129.0,70.0,46.0,31.0,20.0,11.0,16.0,12.0,10.0,149.0,80.0,54.0,26.0,19.0,7.0,15.0
7750,2015-560,US Open,Hard,128.0,G,2015-08-31,4.0,104312.0,25.0,NaN,Andreas Seppi,R,190.0,ITA,31.523614,25.0,1430.0,126205.0,NaN,Q,Tommy Paul,U,NaN,USA,18.288843,437.0,93.0,6-4 6-0 7-5,5.0,R128,NaN,6.0,6.0,83.0,42.0,31.0,22.0,14.0,4.0,7.0,0.0,2.0,88.0,53.0,29.0,12.0,14.0,5.0,13.0
7751,2015-560,US Open,Hard,128.0,G,2015-08-31,5.0,105138.0,23.0,NaN,Roberto Bautista Agut,R,NaN,ESP,27.378508,23.0,1510.0,105732.0,NaN,WC,Pierre Hugues Herbert,R,NaN,FRA,24.454483,92.0,551.0,6-3 6-2 7-6(7),5.0,R128,NaN,4.0,5.0,104.0,59.0,44.0,32.0,15.0,5.0,5.0,17.0,3.0,103.0,54.0,39.0,23.0,14.0,2.0,5.0


In [39]:
sub_df['round'].unique()

array(['R128', 'R64', 'R32', 'R16', 'QF', 'SF', 'F'], dtype=object)

In [41]:
sub_qf = sub_df[sub_df['round'] == 'QF'].copy()

In [66]:
sub_df[sub_df.winner_name == 'Novak Djokovic']

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,winner_rank,winner_rank_points,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,loser_rank,loser_rank_points,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
7747,2015-560,US Open,Hard,128.0,G,2015-08-31,1.0,104925.0,1.0,NaN,Novak Djokovic,R,188.0,SRB,28.276523,1.0,14865.0,105154.0,NaN,NaN,Joao Souza,R,193.0,BRA,27.260780,91.0,554.0,6-1 6-1 6-1,5.0,R128,NaN,8.0,0.0,52.0,39.0,36.0,8.0,11.0,0.0,0.0,3.0,2.0,57.0,31.0,17.0,7.0,10.0,3.0,10.0
7811,2015-560,US Open,Hard,128.0,G,2015-08-31,65.0,104925.0,1.0,NaN,Novak Djokovic,R,188.0,SRB,28.276523,1.0,14865.0,104890.0,NaN,NaN,Andreas Haider Maurer,R,190.0,AUT,28.443532,52.0,870.0,6-4 6-1 6-2,5.0,R64,NaN,5.0,2.0,64.0,42.0,35.0,17.0,13.0,0.0,0.0,3.0,6.0,69.0,32.0,19.0,17.0,12.0,5.0,10.0
7843,2015-560,US Open,Hard,128.0,G,2015-08-31,97.0,104925.0,1.0,NaN,Novak Djokovic,R,188.0,SRB,28.276523,1.0,14865.0,104312.0,25.0,NaN,Andreas Seppi,R,190.0,ITA,31.523614,25.0,1430.0,6-3 7-5 7-5,5.0,R32,NaN,11.0,3.0,91.0,61.0,47.0,17.0,17.0,5.0,7.0,5.0,3.0,109.0,59.0,40.0,26.0,16.0,7.0,12.0
7859,2015-560,US Open,Hard,128.0,G,2015-08-31,113.0,104925.0,1.0,NaN,Novak Djokovic,R,188.0,SRB,28.276523,1.0,14865.0,105138.0,23.0,NaN,Roberto Bautista Agut,R,NaN,ESP,27.378508,23.0,1510.0,6-3 4-6 6-4 6-3,5.0,R16,NaN,8.0,2.0,123.0,75.0,52.0,25.0,19.0,4.0,8.0,4.0,2.0,117.0,69.0,40.0,24.0,19.0,7.0,14.0
7867,2015-560,US Open,Hard,128.0,G,2015-08-31,121.0,104925.0,1.0,NaN,Novak Djokovic,R,188.0,SRB,28.276523,1.0,14865.0,103852.0,18.0,NaN,Feliciano Lopez,L,188.0,ESP,33.943874,19.0,1665.0,6-1 3-6 6-3 7-6(2),5.0,QF,NaN,9.0,0.0,117.0,68.0,52.0,33.0,19.0,5.0,6.0,13.0,6.0,116.0,60.0,46.0,30.0,18.0,5.0,8.0
7871,2015-560,US Open,Hard,128.0,G,2015-08-31,125.0,104925.0,1.0,NaN,Novak Djokovic,R,188.0,SRB,28.276523,1.0,14865.0,105227.0,9.0,NaN,Marin Cilic,R,198.0,CRO,26.921287,9.0,3550.0,6-0 6-1 6-2,5.0,SF,NaN,3.0,1.0,56.0,37.0,28.0,15.0,11.0,0.0,1.0,4.0,5.0,66.0,30.0,18.0,8.0,10.0,6.0,14.0
7873,2015-560,US Open,Hard,128.0,G,2015-08-31,127.0,104925.0,1.0,NaN,Novak Djokovic,R,188.0,SRB,28.276523,1.0,14865.0,103819.0,2.0,NaN,Roger Federer,R,185.0,SUI,34.061602,2.0,9065.0,6-4 5-7 6-4 6-4,5.0,F,NaN,3.0,5.0,155.0,96.0,63.0,32.0,21.0,19.0,23.0,11.0,5.0,137.0,87.0,62.0,23.0,21.0,7.0,13.0


In [68]:
cols = ['match_num', 'winner_id',
       'winner_name','winner_ht', 'winner_age',
       'winner_rank', 'loser_id','loser_name', 'loser_ht',
       'loser_age', 'loser_rank', 'minutes']

In [69]:
sub_df_sub = sub_df[cols]

In [74]:
sub_df_sub[sub_df_sub.match_num == 1]

,match_num,winner_id,winner_name,winner_ht,winner_age,winner_rank,loser_id,loser_name,loser_ht,loser_age,loser_rank,minutes
7747,1.0,104925.0,Novak Djokovic,188.0,28.276523,1.0,105154.0,Joao Souza,193.0,27.26078,91.0,NaN


In [76]:
pd.melt(sub_df_sub[sub_df_sub.match_num == 1], id_vars=['match_num', 'winner_name',	'winner_ht',	'winner_age',	'winner_rank',	'loser_id',	'loser_name',	'loser_ht',	'loser_age',	'loser_rank'])

,match_num,winner_name,winner_ht,winner_age,winner_rank,loser_id,loser_name,loser_ht,loser_age,loser_rank,variable,value
0,1.0,Novak Djokovic,188.0,28.276523,1.0,105154.0,Joao Souza,193.0,27.26078,91.0,winner_id,104925.0
1,1.0,Novak Djokovic,188.0,28.276523,1.0,105154.0,Joao Souza,193.0,27.26078,91.0,minutes,NaN


In [304]:
df.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'winner_rank', 'winner_rank_points', 'loser_id', 'loser_seed',
       'loser_entry', 'loser_name', 'loser_hand', 'loser_ht', 'loser_ioc',
       'loser_age', 'loser_rank', 'loser_rank_points', 'score', 'best_of',
       'round', 'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon',
       'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df',
       'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved',
       'l_bpFaced'],
      dtype='object')

In [307]:
def process_match(series):
    # Define the columns to keep as ID variables
    id_vars = ['match_num', 'tourney_name', 'surface', 'tourney_date', 'minutes']

    # Define the columns to melt into a single variable
    value_vars = ['winner_id', 'winner_name', 'winner_ht', 'winner_age', 'winner_rank', 
                'loser_id', 'loser_name', 'loser_ht', 'loser_age', 'loser_rank']

    # Use the melt function to transform the dataframe
    melted_df = pd.melt(series.to_frame().T, id_vars=id_vars, value_vars=value_vars, var_name='type')

    # Split the 'type' column into separate columns
    melted_df[['player', 'stat']] = melted_df['type'].str.split('_', expand=True)

    types = {
        'match_num': float,
        'minutes': float,
        'tourney_name': str, 
        'surface': str, 
        'type': str,
        'value': object,
        'player': str,
        'stat':str
    }
    melted_df['tourney_date'] = pd.to_datetime(melted_df['tourney_date'])
    melted_df = melted_df.astype(types)
    
    # Pivot the dataframe to reshape it into the desired format
    pivoted_df = melted_df.pivot(index=['match_num', 'tourney_name', 'surface', 'tourney_date', 'minutes', 'player'], 
                                 columns='stat', values='value')

    # # Reset the index of the pivoted dataframe
    pivoted_df = pivoted_df.reset_index()

    # # Rename the columns to match the desired output
    pivoted_df = pivoted_df.rename(columns={
        'id':'id_player',
        'player': 'id'
    })

    pivoted_df[['tourney_name', 'surface', 'tourney_date', 'match_num', 'id', 'name', 'ht', 'age', 'rank', 'minutes']]

    types={
        'match_num': int,
        'id': str,
        'name': str,
        'ht': float,
        'age': float,
        'rank': float,
        'minutes':float,
        'tourney_name': str, 
        'surface': str,
    }

    pivoted_df['tourney_date'] = pd.to_datetime(pivoted_df['tourney_date'])
    # Return the resulting dataframe
    return pivoted_df.astype(types)

In [308]:
process_match(sub_qf.iloc[2])

stat,match_num,tourney_name,surface,tourney_date,minutes,id,age,ht,id_player,name,rank
0,123,US Open,Hard,2015-08-31,NaN,loser,29.286790,203.0,104731.0,Kevin Anderson,14.0
1,123,US Open,Hard,2015-08-31,NaN,winner,30.425736,183.0,104527.0,Stanislas Wawrinka,5.0


In [188]:
# # Initialize the list of transformed DataFrames with a fixed length
# transformed_dfs = [None] * (len(sub_df))

# # Apply the function to each row of the DataFrame and collect the resulting DataFrames in the list
# i = 0
# def collect_transformed_rows(transformed_rows):
#     global i
#     transformed_dfs[i] = transformed_rows
#     i += 1

# sub_df.apply(lambda row: collect_transformed_rows(process_match(row)), axis=1)

# # Concatenate the transformed DataFrames into a single DataFrame
# final_df = pd.concat(transformed_dfs, ignore_index=True)

In [309]:
i = 0

def process_year(dataframe, tourney_name='all'):

    if tourney_name != 'all':
        dataframe = dataframe[dataframe.tourney_name == tourney_name]

    # Initialize the list of transformed DataFrames with a fixed length
    transformed_dfs = [None] * (len(dataframe))

    # Apply the function to each row of the DataFrame and collect the resulting DataFrames in the list
    
    def collect_transformed_rows(transformed_rows):
        global i
        transformed_dfs[i] = transformed_rows
        i += 1
 
    dataframe.apply(lambda row: collect_transformed_rows(process_match(row)), axis=1)

    # # Concatenate the transformed DataFrames into a single DataFrame
    final_df = pd.concat(transformed_dfs, ignore_index=True)

    return final_df

data = process_year(dataframe=df) 

In [310]:
data

stat,match_num,tourney_name,surface,tourney_date,minutes,id,age,ht,id_player,name,rank
0,1,US Open,Hard,2000-08-28,89.0,loser,21.689254,175.0,103297.0,Alex Kim,432.0
1,1,US Open,Hard,2000-08-28,89.0,winner,30.332649,180.0,101736.0,Andre Agassi,1.0
2,2,US Open,Hard,2000-08-28,105.0,loser,21.719370,175.0,103292.0,Gaston Gaudio,26.0
3,2,US Open,Hard,2000-08-28,105.0,winner,22.696783,173.0,103096.0,Arnaud Clement,37.0
4,3,US Open,Hard,2000-08-28,214.0,loser,22.773443,188.0,103082.0,Markus Hantschk,92.0
...,...,...,...,...,...,...,...,...,...,...,...
17521,224,Australian Open,Hard,2017-01-16,184.0,winner,35.441478,185.0,103819.0,Roger Federer,17.0
17522,225,Australian Open,Hard,2017-01-16,296.0,loser,25.672827,188.0,105777.0,Grigor Dimitrov,15.0
17523,225,Australian Open,Hard,2017-01-16,296.0,winner,30.622861,185.0,104745.0,Rafael Nadal,9.0
17524,226,Australian Open,Hard,2017-01-16,217.0,loser,30.622861,185.0,104745.0,Rafael Nadal,9.0


In [303]:
data.match_num.value_counts()

114    138
113    138
100    138
101    138
102    138
      ... 
157     10
156     10
155     10
154     10
226     10
Name: match_num, Length: 226, dtype: int64

In [ ]:
año, torneo, nombre, edad,altura, ranking_ATP_inicial, minutos_jugados_acumulado_hist,
pj_acum_historicos, pg_acum_historico, pp_acum_historico, titulos_ganados_acum, ultima_modificacion, posicion_final, score